# Deep Q-Network for Business Pricing
## Real-Time Pricing Optimization with Neural Networks

This notebook demonstrates how to apply Deep Q-Networks (DQN) to solve real-world business pricing problems.

### Learning Objectives:
- Understand DQN architecture and components
- Implement experience replay and target networks
- Apply RL to dynamic pricing scenarios
- Analyze business performance metrics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import deque
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

print("✅ Libraries imported successfully!")
print("💰 Ready to optimize pricing with DQN!")

## 1. Pricing Environment

We'll create a realistic pricing environment that simulates market dynamics, competitor responses, and customer demand.

In [ ]:
class PricingEnvironment:
    def __init__(self, base_demand=1000, price_sensitivity=2.0, competitor_factor=0.3):
        self.base_demand = base_demand
        self.price_sensitivity = price_sensitivity
        self.competitor_factor = competitor_factor
        
        # Price range
        self.min_price = 10.0
        self.max_price = 50.0
        self.price_step = 2.0
        self.price_actions = np.arange(self.min_price, self.max_price + self.price_step, self.price_step)
        self.n_actions = len(self.price_actions)
        
        # Market state variables
        self.reset()
        
        # Historical data for state representation
        self.history_length = 5
        self.demand_history = deque(maxlen=self.history_length)
        self.price_history = deque(maxlen=self.history_length)
        self.revenue_history = deque(maxlen=self.history_length)
    
    def reset(self):
        """Reset environment to initial state"""
        self.current_price = 25.0  # Starting price
        self.competitor_price = np.random.uniform(20, 35)
        self.market_trend = np.random.uniform(-0.1, 0.1)  # Market growth/decline
        self.seasonality = 0.0
        self.time_step = 0
        
        # Initialize history
        initial_demand = self._calculate_demand(self.current_price)
        for _ in range(self.history_length):
            self.demand_history.append(initial_demand)
            self.price_history.append(self.current_price)
            self.revenue_history.append(initial_demand * self.current_price)
        
        return self._get_state()
    
    def _calculate_demand(self, price):
        """Calculate demand based on price and market conditions"""
        # Base demand equation: D = base_demand * (1 - price_sensitivity * (price - reference_price) / reference_price)
        reference_price = 25.0
        price_effect = -self.price_sensitivity * (price - reference_price) / reference_price
        
        # Competitor effect
        competitor_effect = self.competitor_factor * (self.competitor_price - price) / price
        
        # Market trend and seasonality
        trend_effect = self.market_trend * self.time_step / 100
        seasonal_effect = self.seasonality * np.sin(2 * np.pi * self.time_step / 52)  # Weekly seasonality
        
        # Calculate final demand
        demand_multiplier = 1 + price_effect + competitor_effect + trend_effect + seasonal_effect
        demand = max(0, self.base_demand * demand_multiplier)
        
        # Add noise
        noise = np.random.normal(0, 0.1 * demand)
        return max(0, demand + noise)
    
    def _update_market_conditions(self):
        """Update competitor prices and market conditions"""
        # Competitor price adjustment (reactive to our pricing)
        price_diff = self.current_price - self.competitor_price
        competitor_adjustment = 0.1 * price_diff + np.random.normal(0, 1)
        self.competitor_price = np.clip(self.competitor_price + competitor_adjustment, 15, 45)
        
        # Market trend evolution
        self.market_trend += np.random.normal(0, 0.01)
        self.market_trend = np.clip(self.market_trend, -0.2, 0.2)
        
        # Seasonality (could represent weekly/monthly patterns)
        self.seasonality = 0.1 * np.sin(2 * np.pi * self.time_step / 20)
    
    def _get_state(self):
        """Get current state representation"""
        state = np.array([
            self.current_price / self.max_price,  # Normalized current price
            self.competitor_price / self.max_price,  # Normalized competitor price
            self.market_trend,  # Market trend
            self.seasonality,  # Seasonality factor
            np.mean(list(self.demand_history)) / self.base_demand,  # Normalized avg demand
            np.std(list(self.demand_history)) / self.base_demand,  # Demand volatility
            np.mean(list(self.revenue_history)) / (self.base_demand * self.max_price),  # Normalized avg revenue
            (self.current_price - np.mean(list(self.price_history))) / self.max_price,  # Price change trend
        ])
        return state
    
    def step(self, action):
        """Take pricing action and return next state, reward, done"""
        # Set new price based on action
        self.current_price = self.price_actions[action]
        
        # Calculate demand and revenue
        demand = self._calculate_demand(self.current_price)
        revenue = demand * self.current_price
        
        # Calculate costs (simplified)
        unit_cost = 8.0  # Fixed unit cost
        total_cost = demand * unit_cost
        profit = revenue - total_cost
        
        # Update history
        self.demand_history.append(demand)
        self.price_history.append(self.current_price)
        self.revenue_history.append(revenue)
        
        # Update market conditions
        self._update_market_conditions()
        self.time_step += 1
        
        # Calculate reward (profit with some constraints)
        reward = profit / 1000  # Scale reward
        
        # Penalty for extreme prices
        if self.current_price < 15 or self.current_price > 40:
            reward -= 5
        
        # Bonus for stable pricing (avoid too frequent changes)
        if len(self.price_history) >= 2:
            price_stability = abs(self.price_history[-1] - self.price_history[-2])
            if price_stability <= 2.0:
                reward += 1
        
        # Episode ends after 100 time steps
        done = self.time_step >= 100
        
        next_state = self._get_state()
        
        info = {
            'demand': demand,
            'revenue': revenue,
            'profit': profit,
            'competitor_price': self.competitor_price
        }
        
        return next_state, reward, done, info

# Test the environment
env = PricingEnvironment()
state = env.reset()
print(f"🏪 Pricing Environment Created!")
print(f"State dimension: {len(state)}")
print(f"Number of price actions: {env.n_actions}")
print(f"Price range: ${env.min_price} - ${env.max_price}")
print(f"Initial state: {state}")

## 2. Deep Q-Network Implementation

Now let's implement the DQN with experience replay and target network.

In [ ]:
class DQN(nn.Module):
    def __init__(self, state_size, action_size, hidden_size=128):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, action_size)
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.95, 
                 epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, buffer_size=10000, batch_size=32):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        
        # Neural networks
        self.q_network = DQN(state_size, action_size)
        self.target_network = DQN(state_size, action_size)
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        
        # Experience replay buffer
        self.memory = deque(maxlen=buffer_size)
        
        # Training metrics
        self.episode_rewards = []
        self.episode_profits = []
        self.losses = []
    
    def remember(self, state, action, reward, next_state, done):
        """Store experience in replay buffer"""
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state, training=True):
        """Choose action using epsilon-greedy policy"""
        if training and np.random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        q_values = self.q_network(state_tensor)
        return np.argmax(q_values.cpu().data.numpy())
    
    def replay(self):
        """Train the model on a batch of experiences"""
        if len(self.memory) < self.batch_size:
            return
        
        batch = random.sample(self.memory, self.batch_size)
        states = torch.FloatTensor([e[0] for e in batch])
        actions = torch.LongTensor([e[1] for e in batch])
        rewards = torch.FloatTensor([e[2] for e in batch])
        next_states = torch.FloatTensor([e[3] for e in batch])
        dones = torch.BoolTensor([e[4] for e in batch])
        
        current_q_values = self.q_network(states).gather(1, actions.unsqueeze(1))
        next_q_values = self.target_network(next_states).max(1)[0].detach()
        target_q_values = rewards + (self.gamma * next_q_values * ~dones)
        
        loss = F.mse_loss(current_q_values.squeeze(), target_q_values)
        
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.losses.append(loss.item())
        
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
    
    def update_target_network(self):
        """Copy weights from main network to target network"""
        self.target_network.load_state_dict(self.q_network.state_dict())

# Create DQN agent
state_size = len(env.reset())
action_size = env.n_actions
agent = DQNAgent(state_size, action_size)

print(f"🤖 DQN Agent Created!")
print(f"Network architecture: {state_size} -> 128 -> 128 -> 128 -> {action_size}")
print(f"Experience buffer size: {len(agent.memory)}/{agent.memory.maxlen}")

## 3. Training the DQN Agent

Let's train our DQN agent to learn optimal pricing strategies.

In [ ]:
def train_dqn_agent(agent, env, episodes=500, target_update_freq=10):
    """Train DQN agent on pricing environment"""
    
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        total_profit = 0
        
        for step in range(100):  # Max steps per episode
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            total_reward += reward
            total_profit += info['profit']
            
            # Train the agent
            agent.replay()
            
            if done:
                break
        
        # Update target network periodically
        if episode % target_update_freq == 0:
            agent.update_target_network()
        
        # Store episode metrics
        agent.episode_rewards.append(total_reward)
        agent.episode_profits.append(total_profit)
        
        # Print progress
        if (episode + 1) % 50 == 0:
            avg_reward = np.mean(agent.episode_rewards[-50:])
            avg_profit = np.mean(agent.episode_profits[-50:])
            print(f"Episode {episode + 1:3d} | Avg Reward: {avg_reward:6.2f} | "
                  f"Avg Profit: ${avg_profit:8.0f} | Epsilon: {agent.epsilon:.3f} | "
                  f"Buffer: {len(agent.memory)}")

# Train the agent
print("🚀 Starting DQN Training for Pricing Optimization...")
print("=" * 70)
train_dqn_agent(agent, env, episodes=300)
print("=" * 70)
print("✅ Training Complete!")

## 4. Performance Analysis

Let's analyze the agent's performance and visualize the results.

In [ ]:
# Performance visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# Training rewards
axes[0, 0].plot(agent.episode_rewards, alpha=0.6, label='Episode Reward')
window = 20
moving_avg = pd.Series(agent.episode_rewards).rolling(window=window).mean()
axes[0, 0].plot(moving_avg, color='red', linewidth=2, label=f'{window}-episode average')
axes[0, 0].set_title('Training Rewards')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Reward')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Training profits
axes[0, 1].plot(agent.episode_profits, alpha=0.6, label='Episode Profit')
profit_avg = pd.Series(agent.episode_profits).rolling(window=window).mean()
axes[0, 1].plot(profit_avg, color='green', linewidth=2, label=f'{window}-episode average')
axes[0, 1].set_title('Business Profit')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Profit ($)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Training loss
if agent.losses:
    axes[0, 2].plot(agent.losses, alpha=0.7)
    loss_avg = pd.Series(agent.losses).rolling(window=50).mean()
    axes[0, 2].plot(loss_avg, color='red', linewidth=2)
    axes[0, 2].set_title('Training Loss')
    axes[0, 2].set_xlabel('Training Step')
    axes[0, 2].set_ylabel('MSE Loss')
    axes[0, 2].grid(True, alpha=0.3)

# Test the trained agent
def test_pricing_agent(agent, env, episodes=10):
    test_results = []
    
    for episode in range(episodes):
        state = env.reset()
        episode_data = []
        
        for step in range(100):
            action = agent.act(state, training=False)  # No exploration
            price = env.price_actions[action]
            next_state, reward, done, info = env.step(action)
            
            episode_data.append({
                'step': step,
                'price': price,
                'competitor_price': info['competitor_price'],
                'demand': info['demand'],
                'revenue': info['revenue'],
                'profit': info['profit']
            })
            
            state = next_state
            if done:
                break
        
        test_results.append(pd.DataFrame(episode_data))
    
    return test_results

# Test the agent
test_results = test_pricing_agent(agent, env, episodes=3)
test_df = test_results[0]  # Use first test episode for visualization

# Price comparison
axes[1, 0].plot(test_df['step'], test_df['price'], 'b-', linewidth=2, label='Our Price')
axes[1, 0].plot(test_df['step'], test_df['competitor_price'], 'r--', linewidth=2, label='Competitor Price')
axes[1, 0].set_title('Price Comparison (Test Episode)')
axes[1, 0].set_xlabel('Time Step')
axes[1, 0].set_ylabel('Price ($)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Demand and revenue
ax1 = axes[1, 1]
ax2 = ax1.twinx()

line1 = ax1.plot(test_df['step'], test_df['demand'], 'g-', linewidth=2, label='Demand')
line2 = ax2.plot(test_df['step'], test_df['revenue'], 'orange', linewidth=2, label='Revenue')

ax1.set_xlabel('Time Step')
ax1.set_ylabel('Demand (units)', color='g')
ax2.set_ylabel('Revenue ($)', color='orange')
ax1.set_title('Demand and Revenue')

# Combine legends
lines = line1 + line2
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper right')
ax1.grid(True, alpha=0.3)

# Profit over time
axes[1, 2].plot(test_df['step'], test_df['profit'], 'purple', linewidth=2)
axes[1, 2].axhline(y=test_df['profit'].mean(), color='red', linestyle='--', 
                   label=f'Average: ${test_df["profit"].mean():.0f}')
axes[1, 2].set_title('Profit Over Time')
axes[1, 2].set_xlabel('Time Step')
axes[1, 2].set_ylabel('Profit ($)')
axes[1, 2].legend()
axes[1, 2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Performance summary
print("📊 Performance Summary:")
print(f"   • Average profit per episode: ${np.mean([df['profit'].sum() for df in test_results]):,.0f}")
print(f"   • Average price: ${np.mean([df['price'].mean() for df in test_results]):.2f}")
print(f"   • Average demand: {np.mean([df['demand'].mean() for df in test_results]):.0f} units")
print(f"   • Total revenue: ${np.mean([df['revenue'].sum() for df in test_results]):,.0f}")
print(f"   • Price volatility: ${np.mean([df['price'].std() for df in test_results]):.2f}")